<a href="https://colab.research.google.com/github/Vjy14/golf-/blob/main/golf_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
dataset_path = os.listdir('/content/drive/MyDrive/golf 2222/train')
label_types = os.listdir('/content/drive/MyDrive/golf 2222/train')
print (label_types)

['wrong', 'correct']


In [2]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/golf 2222/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/golf 2222/train' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

     tag                                         video_name
0  wrong  /content/drive/MyDrive/golf 2222/train/wrong/R...
1  wrong  /content/drive/MyDrive/golf 2222/train/wrong/R...
2  wrong  /content/drive/MyDrive/golf 2222/train/wrong/R...
3  wrong  /content/drive/MyDrive/golf 2222/train/wrong/R...
4  wrong  /content/drive/MyDrive/golf 2222/train/wrong/R...
         tag                                         video_name
102  correct  /content/drive/MyDrive/golf 2222/train/correct...
103  correct  /content/drive/MyDrive/golf 2222/train/correct...
104  correct  /content/drive/MyDrive/golf 2222/train/correct...
105  correct  /content/drive/MyDrive/golf 2222/train/correct...
106  correct  /content/drive/MyDrive/golf 2222/train/correct...


In [3]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

In [5]:
dataset_path = os.listdir('/content/drive/MyDrive/golf 2222/test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/golf 2222/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/golf 2222/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['correct', 'wrong']
Types of activities found:  2
       tag                                         video_name
0  correct  dataset/test/correct/Recording 2024-03-19 0829...
1  correct  dataset/test/correct/Recording 2024-03-19 0829...
2  correct  dataset/test/correct/Recording 2024-03-19 0830...
3  correct  dataset/test/correct/Recording 2024-03-19 0831...
4  correct  dataset/test/correct/Recording 2024-03-19 0831...
      tag                                         video_name
18  wrong  dataset/test/wrong/Recording 2024-03-19 112757...
19  wrong  dataset/test/wrong/Recording 2024-03-19 112816...
20  wrong  dataset/test/wrong/Recording 2024-03-19 112844...
21  wrong  dataset/test/wrong/Recording 2024-03-19 112944...
22  wrong  dataset/test/wrong/Recording 2024-03-19 113120...


In [7]:
!pip install tensorflow-docs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 3.6 MB/s eta 0:00:00


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

In [10]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 107
Total videos for testing: 23


,Unnamed: 0,video_name,tag
10,10,/content/drive/MyDrive/golf 2222/train/wrong/R...,wrong
100,100,/content/drive/MyDrive/golf 2222/train/correct...,correct
5,5,/content/drive/MyDrive/golf 2222/train/wrong/R...,wrong
70,70,/content/drive/MyDrive/golf 2222/train/correct...,correct
95,95,/content/drive/MyDrive/golf 2222/train/correct...,correct
26,26,/content/drive/MyDrive/golf 2222/train/wrong/R...,wrong
32,32,/content/drive/MyDrive/golf 2222/train/wrong/R...,wrong
55,55,/content/drive/MyDrive/golf 2222/train/correct...,correct
17,17,/content/drive/MyDrive/golf 2222/train/wrong/R...,wrong
79,79,/content/drive/MyDrive/golf 2222/train/correct...,correct


In [11]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [12]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 1s 0us/step


In [13]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['correct', 'wrong']


array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [14]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [17]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

1/1 [==============================] - 0s 132ms/step
Frame features in train set: (107, 20, 2048)
Frame masks in train set: (107, 20)
train_labels in train set: (107, 1)
test_labels in train set: (23, 1)


In [18]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model
EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
2/3 [===================>..........] - ETA: 0s - loss: 0.5973 - accuracy: 0.7500 
Epoch 1: val_loss improved from inf to 1.81582, saving model to ./tmp/video_classifier
3/3 [==============================] - 12s 1s/step - loss: 0.5515 - accuracy: 0.7838 - val_loss: 1.8158 - val_accuracy: 0.0000e+00
Epoch 2/30
3/3 [==============================] - ETA: 0s - loss: 0.7148 - accuracy: 0.7162
Epoch 2: val_loss improved from 1.81582 to 1.54918, saving model to ./tmp/video_classifier
3/3 [==============================] - 0s 66ms/step - loss: 0.7148 - accuracy: 0.7162 - val_loss: 1.5492 - val_accuracy: 0.0000e+00
Epoch 3/30
3/3 [==============================] - ETA: 0s - loss: 0.5307 - accuracy: 0.7297
Epoch 3: val_loss improved from 1.54918 to 0.83140, saving model to ./tmp/video_classifier
3/3 [==============================] - 0s 69ms/step - loss: 0.5307 - accuracy: 0.7297 - val_loss: 0.8314 - val_accuracy: 0.4545
Epoch 4/30
3/3 [==============================] - ETA: 0s - los

In [24]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(video_path):  # Modified to accept a custom video path
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(video_path)  # Load the video from the provided path
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

# Provide the custom video path here
custom_video_path = '/content/drive/MyDrive/golf 2222/train/wrong/Recording 2024-03-19 103436.mp4'

test_frames = sequence_prediction(custom_video_path)  # Call the function with the custom video path
print(f"Custom video path: {custom_video_path}")


1/1 [==============================] - 0s 22ms/step
  wrong: 84.22%
  correct: 15.78%
Custom video path: /content/drive/MyDrive/golf 2222/train/wrong/Recording 2024-03-19 103436.mp4


In [26]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(video_path):  # Modified to accept a custom video path
    frames = load_video(video_path)  # Load the video from the provided path
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    # Predict class label (assuming binary classification)
    predicted_class = "correct" if probabilities[0] >= 0.5 else "wrong"
    return predicted_class

# Provide the custom video path here
custom_video_path = '/content/drive/MyDrive/golf 2222/train/wrong/Recording 2024-03-19 103436.mp4'

predicted_class = sequence_prediction(custom_video_path)  # Call the function with the custom video path
print(f"Predicted class for video : {predicted_class}")


1/1 [==============================] - 0s 23ms/step
Predicted class for video : wrong
